In [ ]:
import geemap
import ipywidgets as widgets 
import ipyleaflet
from ipywidgets import AppLayout
import cv2
import ee
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import display, clear_output, HTML

In [ ]:
Map = geemap.Map()
#Map

In [ ]:
def maskS2clouds(image):
    qa = image.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
               qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask).divide(10000) 

In [ ]:
#date
dateBox = widgets.DatePicker(
    description = '',
    disabled = False
)

#Map
mapWidget = widgets.Output()
    
#labels
step1 = widgets.Label('Step 1: Select the date')
step2 = widgets.Label('Step 2: Select Region of Interest')
step3 = widgets.Label('Step 3: Model Prediction')
step4 = widgets.Label('Step 4: Display Output')

In [ ]:
getROI = widgets.Button(description='OK')
#display(getROI)
estimate_plastics_Btn = widgets.Button(description='Detect Plastic')
loadModel_Btn = widgets.Button(description='Load/Predict Model')
sat_Btn = widgets.Button(description='Satellite Image')

#Progress bar
progressBar = widgets.IntProgress(
    value=0,
    min=0,
    max=19,
    step=1,
    description='',
    bar_style='info', 
    orientation='horizontal'
)

#Text Area
estimate_plastics_Out = widgets.Textarea(
    value='',
    placeholder='',
    description='',
    disabled=True,
    layout={'height': '300px'}
)

#Display prediction image (Matplotlib plot)
roi_out = widgets.Output()
load_model = widgets.Output()
#display_sat_img = widgets.Ouput()
estimate_plastics_plot = widgets.Output()
sat_out = widgets.Output()
out1 = widgets.Output()
out2 = widgets.Output()
out3 = widgets.Output()
out4 = widgets.Output()

In [ ]:
def plotSatResult(prediction):
    plt.imshow(prediction)
    plt.title("Raw Satellite Image")
    plt.axis("off")
    #display output
    plt.show()
    
def plotPlasticImage(prediction):
    current_cmap = plt.cm.get_cmap()
    current_cmap.set_bad(color='red')
    plt.imshow(((np.array(prediction.values.tolist()).astype('float'))).reshape(40,40))
    plt.title("{}% of Plastics dectected".format(plastic_per))
    plt.axis("off")
    #display output
    plt.show()

In [ ]:
def getStatistics(change):
    
    feature = Map.draw_last_feature
    roi = feature.geometry()
    roi.getInfo()
    #print('updated roi')
    #print(roi.getInfo(), year, month, day)
    
    area = roi.area(1).getInfo()   
    
    year = dateBox.value.year
    month = dateBox.value.month
    day = dateBox.value.day
    #print(year, month, day)
    strt_date = str(year) + "-" + str(month) + "-" + str(day)
    end_date = str(year) + "-" + str(month) + "-" + str((day+5))
    #print(strt_date, end_date)
    #get the data from gee
    # Load Sentinel-2 TOA reflectance data.
    imageCollection = ee.ImageCollection('COPERNICUS/S2') \
        .filterDate(strt_date, end_date) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)) \
        .map(maskS2clouds) \
        .filterBounds(roi) \
        .sort('CLOUDY_PIXEL_PERCENTAGE')
    
    count = imageCollection.size()
    rawImg = ee.Image(imageCollection.toList(imageCollection.size()).get(0))
    
    global B8_image, B4_image, B3_image, B6_image, B11_image
    
    #get bands
    B8_image = geemap.ee_to_numpy(rawImg.select(['B8']), region=roi)
    B4_image = geemap.ee_to_numpy(rawImg.select(['B4']), region=roi)
    B3_image = geemap.ee_to_numpy(rawImg.select(['B3']), region=roi)
    B6_image = geemap.ee_to_numpy(rawImg.select(['B6']), region=roi)
    B11_image = geemap.ee_to_numpy(rawImg.select(['B11']), region=roi)
    
    #with roi_out:
    with out1:
        clear_output()
        print("Image Metadata:")
        print("Accquistion Date:",strt_date)
        print("Count:",count.getInfo())
        print("ROI Area:{} sq m".format(round(area,2)))

getROI.on_click(getStatistics)

In [ ]:
def getlmodel(changed): 
    
    global ndvi, B4_Matrix, ndwi, pi, fdi
    #set bands to same resolution
    targetSize = (40,40)
    B8_Matrix = cv2.resize(B8_image,targetSize,interpolation=cv2.INTER_NEAREST)
    B4_Matrix = cv2.resize(B4_image,targetSize,interpolation=cv2.INTER_NEAREST)
    B3_Matrix = cv2.resize(B3_image,targetSize,interpolation=cv2.INTER_NEAREST)
    B6_Matrix = cv2.resize(B6_image,targetSize,interpolation=cv2.INTER_NEAREST)
    B11_Matrix = cv2.resize(B11_image,targetSize,interpolation=cv2.INTER_NEAREST)
    
    ndvi = (B8_Matrix - B4_Matrix)/(B8_Matrix + B4_Matrix)
    ndwi = (B3_Matrix - B8_Matrix)/(B3_Matrix + B8_Matrix)
    pi = B8_Matrix/(B8_Matrix + B4_Matrix)
    fdi = B8_Matrix - (B6_Matrix + (B11_Matrix - B6_Matrix) * 1.77703)
    
    with out2:
        clear_output()
        print("Model Loaded")
  
    '''
    d = c + 10
    print(d)
    '''
loadModel_Btn.on_click(getlmodel)

In [ ]:
def displaysat(changes):
    with out3:
        clear_output() 
        #print(df["NDVI"].isnull().sum())
        plotSatResult(B4_Matrix)
        
def displayPlot(changes):
    
    global plastic_per
    
    df = pd.DataFrame(data = ndvi.flatten(), columns = ["NDVI"])
    df["NDWI"] = ndwi.flatten()
    df["PI"] = pi.flatten()
    df["FDI"] = fdi.flatten()
    
    def labelling(row):
        if row["NDVI"] > -0.4 and row["NDVI"] < -0.07:
            val = "water"
        elif row["NDVI"] > 0.4 and row["NDVI"] < 0.6:
            val = "seaweed"
        elif row["NDVI"] > 0 and row["NDVI"] < 0.4:
            val = "timber"
        elif row["NDVI"] > -0.07 and row["NDVI"] < 0:
            val = "plastic"
        #elif row["NDVI"] < -0.1 and row["NDVI"] > -0.4:
         #   val = "foam"

        return val

    df["Label"] = df.apply(labelling, axis = 1)
    
    df.loc[df["Label"] == "plastic", "NDVI"] = np.nan
    df.loc[df["Label"] == "plastic", "NDWI"] = np.nan
    df.loc[df["Label"] == "plastic", "PI"] = np.nan
    df.loc[df["Label"] == "plastic", "FDI"] = np.nan
    
    plastic_per = round((df["NDVI"].isnull().sum()/len(df))*100,2)
    #with estimate_plastics_plot:
        #plotSatResult(B4_Matrix)
        
    with out4:
        clear_output() 
        #print(df["NDVI"].isnull().sum())
        #plotSatResult(B4_Matrix)
        plotPlasticImage(df["NDVI"])

sat_Btn.on_click(displaysat)
estimate_plastics_Btn.on_click(displayPlot)

In [ ]:
#Visualize the prediction
#with estimate_plastics_plot:
 #     plotResult(prediction)

In [ ]:
#display map
with mapWidget:
    display(Map)

In [ ]:
#create app layout
#Arrange the layout
vBox = mapWidget
vBox1 = widgets.VBox([step1, dateBox,vBox])
vBox2 = widgets.VBox([step2, getROI])

vBox2 = widgets.VBox([step2,getROI,out1,
                      step3,loadModel_Btn,out2,                            
                      step4,sat_Btn,estimate_plastics_Btn,
                      out3,out4])

AppLayout(header=None,
          left_sidebar=vBox1,
          right_sidebar=vBox2,
          footer=None,
          height="70%", width="150%")